# Kenya Digital Farm Twin - Nitrogen Limited Model

## WOFOST 8.1 with Nitrogen + Water Limitation

This notebook uses **Wofost81_NWLP_MLWB_CNB** which simulates:
- **N**itrogen and **W**ater **L**imited **P**roduction
- **M**ulti-**L**ayer **W**ater **B**alance
- **C**arbon **N**itrogen **B**alance

### ⚠️ Note: Maize not available in WOFOST 8.1
Available crops include: barley, wheat, soybean, potato, rice, cassava, cowpea, groundnut, etc.

---

## 🔧 CONFIGURATION - CHANGE CROP HERE!

In [ ]:
# =============================================================================
# 🌾 CROP CONFIGURATION - CHANGE THESE VALUES TO SWITCH CROPS
# =============================================================================

# Available crops and their Kenya relevance:
CROP_OPTIONS = {
    # CEREALS (similar to maize in N response)
    'barley': {
        'variety': 'Spring_barley_301',
        'kenya_relevance': 'Highland areas, similar N response to maize',
        'planting_month': 3,  # March (long rains)
        'planting_day': 15,
        'season_days': 120,
        'n_demand': 'medium',
    },
    'wheat': {
        'variety': 'Winter_wheat_101',
        'kenya_relevance': 'Narok, Nakuru wheat belt',
        'planting_month': 6,  # June (cool season)
        'planting_day': 1,
        'season_days': 150,
        'n_demand': 'medium-high',
        'needs_vern_override': True,  # Kenya not cold enough!
    },
    'rice': {
        'variety': 'Rice_IR72',
        'kenya_relevance': 'Mwea, Ahero irrigation schemes',
        'planting_month': 3,
        'planting_day': 15,
        'season_days': 140,
        'n_demand': 'high',
    },
    
    # LEGUMES (fix nitrogen, lower N demand)
    'soybean': {
        'variety': 'Soybean_901',
        'kenya_relevance': 'Western Kenya, rotation crop',
        'planting_month': 3,
        'planting_day': 15,
        'season_days': 120,
        'n_demand': 'low',
    },
    'cowpea': {
        'variety': 'Cowpea_VanHeemst_1988',
        'kenya_relevance': 'Semi-arid Eastern Kenya, drought tolerant',
        'planting_month': 10,  # Short rains
        'planting_day': 15,
        'season_days': 90,
        'n_demand': 'low',
    },
    'groundnut': {
        'variety': 'Groundnut_VanHeemst_1988',
        'kenya_relevance': 'Western Kenya, cash crop',
        'planting_month': 3,
        'planting_day': 15,
        'season_days': 120,
        'n_demand': 'low',
    },
    
    # ROOT/TUBER CROPS
    'potato': {
        'variety': 'Potato_701',
        'kenya_relevance': 'Central highlands (Nyandarua, Meru)',
        'planting_month': 3,
        'planting_day': 15,
        'season_days': 120,
        'n_demand': 'high',
    },
    'cassava': {
        'variety': 'Cassava_VanHeemst_1988',
        'kenya_relevance': 'Coast, Western Kenya',
        'planting_month': 3,
        'planting_day': 15,
        'season_days': 300,
        'n_demand': 'low',
    },
    'sweetpotato': {
        'variety': 'Sweetpotato_VanHeemst_1988',
        'kenya_relevance': 'Food security crop, Western Kenya',
        'planting_month': 3,
        'planting_day': 15,
        'season_days': 150,
        'n_demand': 'low',
    },
}

# ════════════════════════════════════════════════════════════════════════════
# ▶▶▶ SELECT YOUR CROP HERE ◀◀◀
# ════════════════════════════════════════════════════════════════════════════

SELECTED_CROP = 'barley'  # <-- CHANGE THIS! Options: barley, wheat, rice, soybean, cowpea, groundnut, potato, cassava, sweetpotato

# ════════════════════════════════════════════════════════════════════════════

# Get crop config
CROP_CONFIG = CROP_OPTIONS[SELECTED_CROP]

print("="*70)
print(f"SELECTED CROP: {SELECTED_CROP.upper()}")
print("="*70)
print(f"Variety: {CROP_CONFIG['variety']}")
print(f"Kenya relevance: {CROP_CONFIG['kenya_relevance']}")
print(f"Planting: Month {CROP_CONFIG['planting_month']}, Day {CROP_CONFIG['planting_day']}")
print(f"Season length: {CROP_CONFIG['season_days']} days")
print(f"N demand: {CROP_CONFIG['n_demand']}")
if CROP_CONFIG.get('needs_vern_override'):
    print("⚠️  Requires vernalization override for Kenya!")

In [ ]:
# =============================================================================
# 📍 LOCATION CONFIGURATION
# =============================================================================

LOCATION_OPTIONS = {
    'trans_nzoia': {'name': 'Trans Nzoia (Kitale)', 'lat': 1.0167, 'lon': 35.0000, 'best_for': ['barley', 'wheat', 'potato']},
    'narok': {'name': 'Narok', 'lat': -1.0833, 'lon': 35.8667, 'best_for': ['wheat', 'barley']},
    'mwea': {'name': 'Mwea (Kirinyaga)', 'lat': -0.7333, 'lon': 37.3500, 'best_for': ['rice']},
    'busia': {'name': 'Busia (Western)', 'lat': 0.4608, 'lon': 34.1108, 'best_for': ['soybean', 'groundnut', 'cassava']},
    'machakos': {'name': 'Machakos (Eastern)', 'lat': -1.5177, 'lon': 37.2634, 'best_for': ['cowpea', 'sorghum']},
    'nyandarua': {'name': 'Nyandarua (Central)', 'lat': -0.4000, 'lon': 36.5000, 'best_for': ['potato', 'wheat']},
    'kilifi': {'name': 'Kilifi (Coast)', 'lat': -3.6305, 'lon': 39.8499, 'best_for': ['cassava', 'cowpea']},
}

# ════════════════════════════════════════════════════════════════════════════
# ▶▶▶ SELECT YOUR LOCATION HERE ◀◀◀
# ════════════════════════════════════════════════════════════════════════════

SELECTED_LOCATION = 'trans_nzoia'  # <-- CHANGE THIS!

# ════════════════════════════════════════════════════════════════════════════

LOCATION = LOCATION_OPTIONS[SELECTED_LOCATION]

print(f"\nLocation: {LOCATION['name']}")
print(f"Coordinates: {LOCATION['lat']}, {LOCATION['lon']}")
print(f"Best crops for this area: {', '.join(LOCATION['best_for'])}")

In [ ]:
# =============================================================================
# 🗓️ YEAR CONFIGURATION
# =============================================================================

SIMULATION_YEAR = 2023  # <-- CHANGE THIS for different weather year

print(f"Simulation year: {SIMULATION_YEAR}")

---
## 1. Setup and Imports

In [ ]:
import pcse
from pcse.models import Wofost81_NWLP_MLWB_CNB
from pcse.input import NASAPowerWeatherDataProvider, YAMLCropDataProvider
from pcse.base import ParameterProvider
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from datetime import date, timedelta

# Style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.size'] = 10

print(f"PCSE version: {pcse.__version__}")
print(f"Model: Wofost81_NWLP_MLWB_CNB (Nitrogen + Water Limited)")

In [ ]:
# Load weather data
print(f"\nLoading weather for {LOCATION['name']}...")
weather = NASAPowerWeatherDataProvider(
    latitude=LOCATION['lat'],
    longitude=LOCATION['lon']
)
print("Done!")

In [ ]:
# Verify crop is available
print("\nVerifying crop availability...")
cropd_test = YAMLCropDataProvider(Wofost81_NWLP_MLWB_CNB)

try:
    cropd_test.set_active_crop(SELECTED_CROP, CROP_CONFIG['variety'])
    print(f"✓ {SELECTED_CROP} / {CROP_CONFIG['variety']} is available!")
except Exception as e:
    print(f"✗ Error: {e}")
    print("\nAvailable crops:")
    cropd_test.print_crops_varieties()

---
## 2. Soil and Site Parameters

In [ ]:
# =============================================================================
# SOIL DATA: Configurable based on Kenya soil types
# =============================================================================

SOIL_TYPES = {
    'nitisol': {  # Trans Nzoia, Eldoret - good for cereals
        'SM0': 0.45, 'SMFCF': 0.36, 'SMW': 0.15, 'CRAIRC': 0.06,
        'RDMSOL': 120.0, 'K0': 10.0, 'SOPE': 1.0, 'KSUB': 1.0,
        'NSOILBASE': 60.0, 'description': 'Highland clay-loam (good)',
    },
    'andosol': {  # Mt. Kenya, Meru - volcanic, very fertile
        'SM0': 0.50, 'SMFCF': 0.40, 'SMW': 0.12, 'CRAIRC': 0.05,
        'RDMSOL': 150.0, 'K0': 20.0, 'SOPE': 2.0, 'KSUB': 1.5,
        'NSOILBASE': 100.0, 'description': 'Volcanic soil (excellent)',
    },
    'vertisol': {  # Narok - black cotton, for wheat
        'SM0': 0.50, 'SMFCF': 0.45, 'SMW': 0.25, 'CRAIRC': 0.04,
        'RDMSOL': 80.0, 'K0': 2.0, 'SOPE': 0.5, 'KSUB': 0.3,
        'NSOILBASE': 50.0, 'description': 'Black cotton clay (wheat)',
    },
    'ferralsol': {  # Western Kenya - leached
        'SM0': 0.42, 'SMFCF': 0.32, 'SMW': 0.14, 'CRAIRC': 0.06,
        'RDMSOL': 100.0, 'K0': 8.0, 'SOPE': 1.0, 'KSUB': 0.8,
        'NSOILBASE': 35.0, 'description': 'Tropical red (low N)',
    },
    'luvisol': {  # Semi-arid Eastern
        'SM0': 0.38, 'SMFCF': 0.25, 'SMW': 0.10, 'CRAIRC': 0.08,
        'RDMSOL': 80.0, 'K0': 15.0, 'SOPE': 2.0, 'KSUB': 1.0,
        'NSOILBASE': 25.0, 'description': 'Semi-arid sandy-loam (very low N)',
    },
}

# ════════════════════════════════════════════════════════════════════════════
# ▶▶▶ SELECT YOUR SOIL TYPE HERE ◀◀◀
# ════════════════════════════════════════════════════════════════════════════

SELECTED_SOIL = 'nitisol'  # <-- CHANGE THIS! Options: nitisol, andosol, vertisol, ferralsol, luvisol

# ════════════════════════════════════════════════════════════════════════════

soil_base = SOIL_TYPES[SELECTED_SOIL]

soildata = {
    # Water balance
    'SM0': soil_base['SM0'],
    'SMFCF': soil_base['SMFCF'],
    'SMW': soil_base['SMW'],
    'CRAIRC': soil_base['CRAIRC'],
    'RDMSOL': soil_base['RDMSOL'],
    'K0': soil_base['K0'],
    'SOPE': soil_base['SOPE'],
    'KSUB': soil_base['KSUB'],
    
    # Surface
    'IFUNRN': 0,
    'SSMAX': 0.0,
    'SSI': 0.0,
    'WAV': 50.0,
    'NOTINF': 0.0,
    'SMLIM': soil_base['SMFCF'],
    
    # NITROGEN
    'NSOILBASE': soil_base['NSOILBASE'],
    'NSOILBASE_FR': 0.025,
}

print(f"\nSoil type: {SELECTED_SOIL.upper()}")
print(f"Description: {soil_base['description']}")
print(f"Soil N pool: {soil_base['NSOILBASE']} kg N/ha")
print(f"Available water: {(soil_base['SMFCF'] - soil_base['SMW'])*100:.1f}% by volume")

In [ ]:
# Site data
sitedata = {
    'CO2': 415.0,
    'NAVAILI': 20.0,      # Initial available N (kg/ha)
    'BG_N_SUPPLY': 0.5,   # Background N supply (kg/ha/day)
}

print(f"Initial available N: {sitedata['NAVAILI']} kg/ha")
print(f"Background N supply: {sitedata['BG_N_SUPPLY']} kg/ha/day")

---
## 3. Fertilizer Scenarios

In [ ]:
# Build agromanagement using PROPER DATE OBJECTS (PCSE 6.0 requirement!)
def build_agro(n_applications, year=SIMULATION_YEAR):
    """
    Build agromanagement dictionary for selected crop with specified N applications.
    
    IMPORTANT: PCSE 6.0 requires date objects as keys, not strings!
    
    n_applications: list of (days_after_planting, kg_n, recovery_fraction)
    """
    # Create date objects
    planting_date = date(year, CROP_CONFIG['planting_month'], CROP_CONFIG['planting_day'])
    end_date = date(year, 12, 31)
    max_duration = CROP_CONFIG['season_days']
    
    # Build timed events for N applications
    timed_events = None
    if n_applications:
        timed_events = []
        for i, (days, amount, recovery) in enumerate(n_applications):
            app_date = planting_date + timedelta(days=days)
            timed_events.append({
                'event_signal': 'apply_n',
                'name': f'N application {i+1}',
                'comment': f'{amount} kg N/ha',
                'events_table': [
                    {app_date: {'amount': amount, 'recovery': recovery}}
                ]
            })
    
    # Build agromanagement dictionary with DATE OBJECTS as keys
    agro = {
        'Version': 1.0,
        'AgroManagement': [
            {
                planting_date: {  # <-- This MUST be a date object!
                    'CropCalendar': {
                        'crop_name': SELECTED_CROP,
                        'variety_name': CROP_CONFIG['variety'],
                        'crop_start_date': planting_date,  # date object
                        'crop_start_type': 'emergence',
                        'crop_end_date': end_date,  # date object
                        'crop_end_type': 'earliest',
                        'max_duration': max_duration,
                    },
                    'TimedEvents': timed_events,
                    'StateEvents': None,
                }
            }
        ]
    }
    
    return agro

# Test the function
test_agro = build_agro([(0, 20, 0.7), (30, 15, 0.6)])
print("Agromanagement structure test:")
print(f"  Campaign date type: {type(list(test_agro['AgroManagement'][0].keys())[0])}")
print(f"  Campaign date value: {list(test_agro['AgroManagement'][0].keys())[0]}")
print("  ✓ Using proper date objects!")

In [ ]:
# Define fertilizer scenarios
# Format: (days_after_planting, kg_N, recovery_fraction)

FERT_SCENARIOS = {
    'none': {
        'name': 'No Fertilizer',
        'total_n': 0,
        'applications': [],
        'description': 'Baseline - soil N only',
    },
    'low': {
        'name': 'Low (25 kg N/ha)',
        'total_n': 25,
        'applications': [(0, 15, 0.7), (30, 10, 0.6)],
        'description': 'Typical smallholder',
    },
    'medium': {
        'name': 'Medium (50 kg N/ha)',
        'total_n': 50,
        'applications': [(0, 20, 0.7), (30, 15, 0.65), (50, 15, 0.6)],
        'description': 'Improved smallholder',
    },
    'recommended': {
        'name': 'Recommended (75 kg N/ha)',
        'total_n': 75,
        'applications': [(0, 25, 0.7), (30, 25, 0.65), (50, 25, 0.6)],
        'description': 'Extension recommendation',
    },
    'high': {
        'name': 'High (100 kg N/ha)',
        'total_n': 100,
        'applications': [(0, 30, 0.7), (25, 35, 0.65), (50, 35, 0.6)],
        'description': 'Commercial/intensive',
    },
}

print("Fertilizer scenarios defined:")
for key, scenario in FERT_SCENARIOS.items():
    print(f"  {key}: {scenario['name']} - {scenario['description']}")

---
## 4. Run Simulations

In [ ]:
def run_simulation(fert_scenario_key):
    """Run simulation for a given fertilizer scenario."""
    scenario = FERT_SCENARIOS[fert_scenario_key]
    print(f"\nRunning: {scenario['name']}...")
    
    # Build agromanagement with proper date objects
    agro = build_agro(scenario['applications'])
    
    # Fresh crop data
    cropd = YAMLCropDataProvider(Wofost81_NWLP_MLWB_CNB)
    cropd.set_active_crop(SELECTED_CROP, CROP_CONFIG['variety'])
    
    # Create parameters
    params = ParameterProvider(
        cropdata=cropd,
        soildata=soildata,
        sitedata=sitedata
    )
    
    # Apply vernalization override if needed (for wheat)
    if CROP_CONFIG.get('needs_vern_override'):
        print("  Applying vernalization override...")
        params.set_override('VERNSAT', 0)
        params.set_override('VERNBASE', 0)
        params.set_override('VERNDVS', 0)
    
    try:
        model = Wofost81_NWLP_MLWB_CNB(params, weather, agro)
        model.run_till_terminate()
        
        output = model.get_output()
        df = pd.DataFrame(output)
        summary = model.get_summary_output()
        
        if summary and len(summary) > 0:
            s = summary[0]
            
            # Get yield - could be TWSO (grains/tubers) or check TAGP
            grain_yield = s.get('TWSO', 0)
            
            # For root crops, may need different handling
            if SELECTED_CROP in ['potato', 'cassava', 'sweetpotato']:
                main_yield = grain_yield if grain_yield > 0 else s.get('TAGP', 0) * 0.5
                yield_type = 'tuber/root'
            else:
                main_yield = grain_yield if grain_yield > 0 else s.get('TAGP', 0) * 0.4
                yield_type = 'grain'
            
            print(f"  ✓ {yield_type.capitalize()} yield: {main_yield:.0f} kg/ha ({main_yield/1000:.2f} t/ha)")
            
            return {
                'df': df,
                'summary': s,
                'yield_kg': main_yield,
                'yield_t': main_yield / 1000,
                'scenario': scenario['name'],
                'n_rate': scenario['total_n'],
                'tagp': s.get('TAGP', 0),
                'laimax': s.get('LAIMAX', 0),
            }
        else:
            print(f"  ✗ No summary output")
            return None
            
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

In [ ]:
# Run all scenarios
print("="*70)
print(f"RUNNING FERTILIZER RESPONSE SCENARIOS FOR {SELECTED_CROP.upper()}")
print(f"Location: {LOCATION['name']}")
print(f"Year: {SIMULATION_YEAR}")
print("="*70)

results = []
dataframes = {}

for key in FERT_SCENARIOS.keys():
    result = run_simulation(key)
    if result:
        results.append(result)
        dataframes[result['scenario']] = result['df']

if results:
    df_results = pd.DataFrame([{
        'scenario': r['scenario'],
        'n_rate': r['n_rate'],
        'yield_kg': r['yield_kg'],
        'yield_t': r['yield_t'],
        'tagp': r['tagp'],
        'laimax': r['laimax'],
    } for r in results])

    print("\n" + "="*70)
    print("RESULTS SUMMARY")
    print("="*70)
    print(df_results.to_string(index=False))
else:
    print("\n⚠️ No successful simulations. Check error messages above.")

---
## 5. Visualizations

In [ ]:
# Figure 1: Nitrogen response curve
if results:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(df_results)))

    # (a) Bar chart
    ax = axes[0]
    bars = ax.bar(range(len(df_results)), df_results['yield_t'], color=colors, 
                  edgecolor='white', linewidth=2)
    ax.set_xticks(range(len(df_results)))
    ax.set_xticklabels(df_results['scenario'], rotation=20, ha='right')

    for bar, val in zip(bars, df_results['yield_t']):
        ax.annotate(f'{val:.2f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                    xytext=(0, 5), textcoords='offset points', ha='center', fontweight='bold')

    ax.set_ylabel('Yield (tonnes/ha)', fontweight='bold')
    ax.set_xlabel('Fertilizer Scenario', fontweight='bold')
    ax.set_title(f'(a) {SELECTED_CROP.capitalize()} Yield by N Rate', fontweight='bold')
    ax.set_ylim(0, max(df_results['yield_t']) * 1.2)
    ax.grid(True, axis='y', alpha=0.3)

    # (b) Response curve
    ax = axes[1]
    ax.plot(df_results['n_rate'], df_results['yield_t'], 'o-', color='#27AE60',
            linewidth=2, markersize=10, markeredgecolor='white', markeredgewidth=2)
    ax.fill_between(df_results['n_rate'], 0, df_results['yield_t'], alpha=0.2, color='#27AE60')

    for _, row in df_results.iterrows():
        ax.annotate(f'{row["yield_t"]:.2f}', xy=(row['n_rate'], row['yield_t']),
                    xytext=(5, 5), textcoords='offset points', fontsize=9)

    ax.set_xlabel('Nitrogen Applied (kg N/ha)', fontweight='bold')
    ax.set_ylabel('Yield (tonnes/ha)', fontweight='bold')
    ax.set_title('(b) Nitrogen Response Curve', fontweight='bold')
    ax.set_xlim(-5, max(df_results['n_rate']) * 1.1)
    ax.set_ylim(0, max(df_results['yield_t']) * 1.2)
    ax.grid(True, alpha=0.3)

    fig.suptitle(f'{SELECTED_CROP.upper()} Nitrogen Response - {LOCATION["name"]} ({SIMULATION_YEAR})',
                 fontsize=14, fontweight='bold', y=1.02)

    plt.tight_layout()
    plt.savefig(f'{SELECTED_CROP}_n_response.png', bbox_inches='tight', facecolor='white', dpi=150)
    plt.show()

    print(f"\n✓ Figure saved as '{SELECTED_CROP}_n_response.png'")
else:
    print("No results to plot.")

In [ ]:
# Figure 2: Growth dynamics comparison
if dataframes:
    fig, axes = plt.subplots(2, 2, figsize=(12, 9))

    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(dataframes)))
    color_map = dict(zip(dataframes.keys(), colors))

    for name, df in dataframes.items():
        c = color_map[name]
        
        # LAI
        if 'LAI' in df.columns:
            axes[0, 0].plot(df['day'], df['LAI'], color=c, linewidth=2, label=name)
        
        # Biomass
        if 'TAGP' in df.columns:
            axes[0, 1].plot(df['day'], df['TAGP']/1000, color=c, linewidth=2, label=name)
        
        # Yield component
        if 'TWSO' in df.columns:
            axes[1, 0].plot(df['day'], df['TWSO']/1000, color=c, linewidth=2, label=name)
        
        # DVS
        if 'DVS' in df.columns:
            axes[1, 1].plot(df['day'], df['DVS'], color=c, linewidth=2, label=name)

    # Format
    axes[0, 0].set_ylabel('LAI (m²/m²)', fontweight='bold')
    axes[0, 0].set_title('(a) Leaf Area Index', fontweight='bold')
    axes[0, 0].legend(loc='upper right', fontsize=8)

    axes[0, 1].set_ylabel('Total Biomass (t/ha)', fontweight='bold')
    axes[0, 1].set_title('(b) Above-ground Biomass', fontweight='bold')

    axes[1, 0].set_ylabel('Yield (t/ha)', fontweight='bold')
    axes[1, 0].set_xlabel('Date', fontweight='bold')
    axes[1, 0].set_title('(c) Yield Accumulation', fontweight='bold')

    axes[1, 1].set_ylabel('DVS', fontweight='bold')
    axes[1, 1].set_xlabel('Date', fontweight='bold')
    axes[1, 1].set_title('(d) Development Stage', fontweight='bold')
    axes[1, 1].axhline(y=1.0, color='orange', linestyle='--', alpha=0.7, label='Flowering')
    axes[1, 1].axhline(y=2.0, color='red', linestyle='--', alpha=0.7, label='Maturity')

    for ax in axes.flat:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.grid(True, alpha=0.3)

    fig.suptitle(f'{SELECTED_CROP.upper()} Growth Under Different N Scenarios', 
                 fontsize=14, fontweight='bold', y=0.98)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f'{SELECTED_CROP}_growth_comparison.png', bbox_inches='tight', facecolor='white', dpi=150)
    plt.show()

    print(f"\n✓ Figure saved as '{SELECTED_CROP}_growth_comparison.png'")
else:
    print("No results to plot.")

---
## 6. Summary

In [ ]:
print("="*70)
print(f"SUMMARY: {SELECTED_CROP.upper()} SIMULATION")
print("="*70)

print(f"\n📍 Configuration:")
print(f"   Crop: {SELECTED_CROP} ({CROP_CONFIG['variety']})")
print(f"   Location: {LOCATION['name']}")
print(f"   Soil: {SELECTED_SOIL} ({soil_base['description']})")
print(f"   Year: {SIMULATION_YEAR}")

if results:
    print(f"\n🌾 Yield Results:")
    print("-"*50)
    for _, row in df_results.iterrows():
        print(f"   {row['scenario']:<25} → {row['yield_t']:.2f} t/ha")

    if len(df_results) >= 2:
        no_fert = df_results[df_results['n_rate'] == 0]['yield_t'].values
        max_fert = df_results['yield_t'].max()
        
        if len(no_fert) > 0 and no_fert[0] > 0:
            improvement = (max_fert - no_fert[0]) / no_fert[0] * 100
            print(f"\n💡 Potential yield increase with fertilizer: +{improvement:.0f}%")

    print(f"\n📈 Figures saved:")
    print(f"   ✓ {SELECTED_CROP}_n_response.png")
    print(f"   ✓ {SELECTED_CROP}_growth_comparison.png")
else:
    print("\n⚠️ No successful simulations.")

print("\n" + "="*70)
print("To change crop, location, or soil: edit the SELECTED_* variables at the top!")
print("="*70)